# Building centroid model

In [ ]:
%config IPCompleter.use_jedi = False
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
from moffat import Moffat2D

## The model

![](../../docs/architecture.png)

In [ ]:
size = 15

n = Moffat2D(size)

model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(size, size, 1), use_bias=True, padding="same"),
    layers.MaxPooling2D((2, 2), padding="same"),
    layers.Conv2D(128, (3, 3), activation='relu', use_bias=True, padding="same"),
    layers.MaxPooling2D((2, 2), padding="same"),
    layers.Conv2D(256, (3, 3), activation='relu', use_bias=True, padding="same"),
    layers.Flatten(),
    layers.Dense(2048, activation="sigmoid", use_bias=True),
    layers.Dense(512, activation="sigmoid", use_bias=True),
    layers.Dense(2),
])

### Training strategy

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=20,
  decay_rate=1,
  staircase=False)

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule), 
    loss=tf.keras.losses.Huber(), 
    metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
# Training

train = 2000
test = 10000
epochs = 80

train_dataset = n.random_model_label(train)
test_dataset = n.random_model_label(test)
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset)
test_dataset = tf.data.Dataset.from_tensor_slices(test_dataset)

BATCH_SIZE = 100
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

model.fit(train_dataset, epochs=epochs, validation_data=test_dataset, )

In [ ]:
# Saving
model.save()